In [ ]:
pip install faiss 
#conda install -c pytorch faiss-cpu

In [ ]:
import nltk
nltk.download('punkt')

In [1]:
import json
import numpy as np
import faiss
import requests
import re
import pandas as pd
resp = requests.get('https://www.sec.gov/files/company_tickers.json')
#df= pd.read_json(resp.text)
df = pd.read_json(resp.text, orient ='index')

# preprocessing data
df0 = df.copy()
df0['title'] = df0['title'].apply(str.lower)
df0['title'] = df0['title'].replace('&', 'and')
df0['title'] = df0['title'].apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', x))
df0['title'] = df0['title'].apply(lambda x: re.sub("\s\s+", " ", x))
df0['title'] = df0['title'].apply(lambda x: x.strip())

In [3]:
from nltk.tokenize import word_tokenize
companies = set(df0["title"].values.tolist())
companies = list(companies)
company_tokens = [set(word for word in word_tokenize(company)) for company in companies]

In [4]:
# Define function to compute character n-gram embeddings
def char_ngram_embeddings(names, n=3, dim=100):
    char_ngrams = set()
    for name in names:
        for i in range(len(name) - n + 1):
            char_ngrams.add(name[i:i+n])
    char_ngram_dict = {ngram: i for i, ngram in enumerate(char_ngrams)}
    embeddings = np.zeros((len(names), len(char_ngrams), dim))
    for i, name in enumerate(names):
        for j in range(len(name) - n + 1):
            ngram = name[j:j+n]
            if ngram in char_ngram_dict:
                embeddings[i, char_ngram_dict[ngram]] += np.random.randn(dim)
    return embeddings

# Compute character n-gram embeddings for company names
embeddings = char_ngram_embeddings(companies)

In [ ]:
# Build Faiss index
index = faiss.IndexFlatIP(embeddings.shape[1])
index.add(embeddings.astype(np.float32))
# 就是ta卡！

In [ ]:
# Define function to perform similarity search
def search(query, companies, index, k=10):
    # Compute character n-gram embedding for query
    query_embedding = char_ngram_embeddings([query])[0]
    # Search Faiss index
    distances, indices = index.search(query_embedding.astype(np.float32), k)
    # Return top-k matching company names
    return [companies[i] for i in indices[0]]

In [ ]:

# Test similarity search on example query
query = 'Google'
matches = search(query, companies, index, k=5)
print(f"Top matches for '{query}':")
for match in matches:
    print(match)